In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from pathlib import Path
from ase.io import read
from mbeml.featurization import (
    generate_ligand_racs_names,
    generate_standard_racs_names,
    sort_connecting_atoms,
    featurize_single_ligand,
)
from molSimplify.Classes.mol3D import mol3D
from molSimplify.Classes.mol2D import Mol2D
from molSimplify.Classes.ligand import ligand_breakdown, ligand_assign_consistent
from molSimplify.Informatics.lacRACAssemble import get_descriptor_vector

In [2]:
raw_data_path = Path("../../raw_data")
data_path = Path("../../data")
data_sets = {
    "training": pd.read_csv(raw_data_path / "training" / "training_data_raw.csv"),
    "validation": pd.read_csv(raw_data_path / "validation" / "validation_data_raw.csv"),
    "ligand_test": pd.read_csv(raw_data_path / "ligand_test" / "ligand_test_data_raw.csv"),
    "composition_test": pd.read_csv(raw_data_path / "composition_test" / "composition_test_data_raw.csv"),
}

In [3]:
# dictionary assigning the molecular graph determinant to a charge per connecting atom
ligand_charges = {
    "6aef01e37dbae6d1036490f90be7fdfb": -1,  # fluoride, graph_det=18.9984
    "3dc71de014d4aa968cdd9f7c1e905feb": -2,  # sulfide, graph_det=32.065
    "481b474701a5d44d8bc0a1051f702ad0": -1,  # chloride, graph_det=35.452999999
    "8c49bdf7d036bdf3178afbdb02b59e5f": -1,  # iodide, graph_det=126.90446999
    "8991adadca2e8cc23e62278399df44a1": -1,  # hydroxyl [OH-], graph_det=-243.9154775
    "09642cb93892875b5495ff15993a6289": -1,  # cyanide, graph_det=-28133.19404
    "608a852d63f3b4f49956bae07e2b3f8c": -2,  # [O-][O-], graph_det=-65270.18935
    "247db616eed8055e98564c9c3d17734e": -1,  # NC[O-], graph_det=-967339.6416
    "1e18033c0b1c215888a09256d7639180": -1,  # ox, graph_det=138760516791
    "0dfc6836e022406f1be7b5627451d590": -0.5,  # acac, graph_det=-2.822252628e+16
    "0bcd1e34289690552dd5b876ac74361e": -0.5,  # porphyrin, graph_det=2.6410120150e+51
}

In [4]:
standard_racs_names = generate_standard_racs_names()
ligand_racs_names = generate_ligand_racs_names()

def featurize_data_set(df_raw, xyzs_dir):
    df_out = df_raw.copy()
    
    for index, row in df_out.iterrows():
        # ############################################# #
        # Use molSimplify to evaluate the standard-RACs #
        # ############################################# #
        mol = mol3D()
        # Use low spin geometry for featurization
        low_spin = 2 if row["high_spin"] % 2 == 0 else 1
        mol.readfromxyz(xyzs_dir / f"{row['name']}_{low_spin}.xyz")
        _, standard_racs = get_descriptor_vector(mol)

        liglist, ligdents, ligcons = ligand_breakdown(mol)
        (
            ax_ligand_list,
            eq_ligand_list,
            _,
            _,
            ax_con_int_list,
            eq_con_int_list,
            _,
            _,
            _,
        ) = ligand_assign_consistent(mol, liglist, ligdents, ligcons, loud=False)
        # These are somewhat difficult to calculate given that there are "fractional" ligands in the ligand_list.
        # This is accounted for by the multipling the fractional ligand charge with the number of connecting atoms in the subset
        axial_charge = sum(
            [
                ligand_charges.get(Mol2D.from_mol3d(lig.mol).graph_hash(), 0) * len(con)
                for lig, con in zip(ax_ligand_list, ax_con_int_list)
            ]
        )
        equatorial_charge = sum(
            [
                ligand_charges.get(Mol2D.from_mol3d(lig.mol).graph_hash(), 0) * len(con)
                for lig, con in zip(eq_ligand_list, eq_con_int_list)
            ]
        )
        standard_racs[1] = axial_charge / len(ax_ligand_list)
        standard_racs[3] = equatorial_charge / len(eq_ligand_list)

        df_out.loc[index, standard_racs_names] = standard_racs
    

        # #################### #
        # evaluate ligand-RACs #
        # #################### #
        ase_atoms = read(xyzs_dir / f"{row['name']}_{low_spin}.xyz")
        mol2D = Mol2D.from_mol3d(mol)
        # Check that there is only one metal and that it is the first atom
        assert mol2D.find_metal() == [0]

        connecting_atoms = sort_connecting_atoms(ase_atoms, list(mol2D.neighbors(0)))

        split_graph = mol2D.copy()
        # Remove edges from center (index 0) to the connecting atoms
        split_graph.remove_edges_from([(0, c) for c in connecting_atoms])
        ligand_graphs = [split_graph.subgraph(gi) for gi in nx.connected_components(split_graph)][1:]
        
        ligand_racs = np.zeros((6, 33))
        for ci, c in enumerate(connecting_atoms):
            ligand_racs[ci, 1:] = featurize_single_ligand(split_graph, c, depth=3)

            # Kazy way of finding correct subgraph for the connecting atom c
            for lig_graph in ligand_graphs:            
                if c in lig_graph:
                    break
            else:
                raise ValueError(f'Unable to find ligand corresponding to connecting atom {c}')
            
            lig_hash = lig_graph.graph_hash()
            if lig_hash in ligand_charges:
                ligcharge = ligand_charges[lig_hash]
            else:
                ligcharge = 0.0
            ligand_racs[ci, 0] = ligcharge
            if lig_graph.graph_determinant() in ligand_charges:
                print(lig_graph.graph_determinant(), lig_graph.graph_hash())
        
        df_out.loc[index, ligand_racs_names] = ligand_racs.flatten()
    return df_out

In [ ]:
for key, df_raw in data_sets.items():
    df = featurize_data_set(df_raw, raw_data_path / key / "xyzs")
    df.to_csv(data_path / f"{key}_data.csv")

In [73]:
df_lig_test = featurize_data_set(data_sets["ligand_test"], raw_data_path / "ligand_test" / "xyzs")

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:29, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:29, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:29, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:29, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:29, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in Perceive

In [64]:
df_lig_test_ref = pd.read_csv(data_path / "ligand_test_data.csv")
pd.testing.assert_frame_equal(df_lig_test, df_lig_test_ref)

In [65]:
df_comp_test = featurize_data_set(data_sets["composition_test"], raw_data_path / "composition_test" / "xyzs")

In [66]:
df_comp_test_ref = pd.read_csv(data_path / "composition_test_data.csv")
pd.testing.assert_frame_equal(df_comp_test, df_comp_test_ref)

In [76]:
df_val = featurize_data_set(data_sets["validation"], raw_data_path / "validation" / "xyzs")

In [ ]:
df_val_ref = pd.read_csv(data_path / "validation_data.csv")
pd.testing.assert_frame_equal(df_val, df_val_ref)

In [ ]:
df_train = featurize_data_set(data_sets["training"], raw_data_path / "training" / "xyzs")

In [57]:
df_train_ref = pd.read_csv(data_path / "training_data.csv")
pd.testing.assert_frame_equal(df_train, df_train_ref)

In [14]:
df_raw = pd.read_csv(raw_data_path / "ligand_test" / "ligand_test_data_raw.csv")
df_lig_test = pd.read_csv("/home/ralf/Dropbox (MIT)/many_body_expansion/data/new_calculations/ligand_generalization_set/ligand_generalization_data.csv")

In [62]:
def ligand_renaming(lig: str):
    if lig == "bipyrimidine":
        return "bidiazine"
    return lig

for index, row in df_raw.iterrows():
    metal, ox, lig1, lig2, lig3, lig4, lig5, lig6 = row["name"].split("_")

    print(metal, ox, lig1)
    mask = (
        (df_lig_test["metal"] == metal)
        & (df_lig_test["ox"] == int(ox))
        & (df_lig_test["lig1"] == ligand_renaming(lig1))
        & (df_lig_test["lig2"] == ligand_renaming(lig2))
        & (df_lig_test["lig3"] == ligand_renaming(lig3))
        & (df_lig_test["lig4"] == ligand_renaming(lig4))
        & (df_lig_test["lig5"] == ligand_renaming(lig5))
        & (df_lig_test["lig6"] == ligand_renaming(lig6))
    )
    assert len(df_lig_test[mask])  == 2
    # Loop over the two spins and write .xyz
    for _, spin in df_lig_test[mask].iterrows():
        with open(raw_data_path / "ligand_test" / "xyzs" / f"{row['name']}_{spin['spin']}.xyz", "w") as fout:
            fout.write(spin["opt_geo"].replace("\t", "    "))

co 2 1H-tetrazole
co 2 1H-triazole
co 2 4H-pyran
co 2 4H-thiopyran
co 2 [NH]=[CH]-[OH]
co 2 [PH2]-[CH2]-[OH]
co 2 [PH2]-[NH]-[NH]-[PH2]
co 2 [PH]=[CH]-[OH]
co 2 [NH2]-[NH]-[NH]-[NH2]
co 2 [NH2]-[O]-[O]-[NH2]
co 2 [OH]-[CH]=[CH]-[OH]
co 2 acrylamide
co 2 bipyrimidine
co 2 bifuran
co 2 dmf
co 2 oxazoline
co 2 pyridine-N-oxide
co 2 thiane
co 2 thioazole
co 2 thioformaldehyde
co 2 thiophene
co 3 1H-tetrazole
co 3 1H-triazole
co 3 [PH2]-[CH2]-[OH]
co 3 acrylamide
co 3 bipyrimidine
co 3 dmf
co 3 oxazoline
co 3 thioazole
fe 2 1H-tetrazole
fe 2 1H-triazole
fe 2 4H-pyran
fe 2 4H-thiopyran
fe 2 [NH]=[CH]-[OH]
fe 2 [PH2]-[CH2]-[OH]
fe 2 [PH2]-[NH]-[NH]-[PH2]
fe 2 [PH]=[CH]-[OH]
fe 2 [NH2]-[NH]-[NH]-[NH2]
fe 2 [NH2]-[O]-[O]-[NH2]
fe 2 [OH]-[CH]=[CH]-[OH]
fe 2 acrylamide
fe 2 bipyrimidine
fe 2 bifuran
fe 2 dmf
fe 2 oxazoline
fe 2 pyridine-N-oxide
fe 2 thiane
fe 2 thioazole
fe 2 thioformaldehyde
fe 2 thiophene
fe 3 1H-tetrazole
fe 3 1H-triazole
fe 3 [NH]=[CH]-[OH]
fe 3 [PH2]-[CH2]-[OH]
fe 3 [PH2]-[N

In [55]:
df_lig_test

,metal,ox,spin,lig1,lig2,lig3,lig4,lig5,lig6,energy,...,metal_spin_target,ss_flag,metal_spin_flag,geo_flag,mol_graph_det,opt_geo,alphaHOMO,alphaLUMO,betaHOMO,betaLUMO
0,co,2,2,1H-tetrazole,1H-tetrazole,1H-tetrazole,1H-tetrazole,1H-tetrazole,1H-tetrazole,-1694.285329,...,1.0,1.0,1.0,1.0,-3.369050e+68,"43\n11/21/2023 17:07, XYZ structure generated ...",-0.4669,-0.3009,-0.4864,-0.2676
1,co,2,4,1H-tetrazole,1H-tetrazole,1H-tetrazole,1H-tetrazole,1H-tetrazole,1H-tetrazole,-1694.294977,...,3.0,1.0,1.0,1.0,-3.369050e+68,"43\n11/21/2023 17:07, XYZ structure generated ...",-0.5012,-0.2471,-0.5024,-0.3237
2,co,2,2,1H-triazole,1H-triazole,1H-triazole,1H-triazole,1H-triazole,1H-triazole,-1598.144214,...,1.0,1.0,1.0,1.0,-8.394981e+63,"49\n11/21/2023 17:07, XYZ structure generated ...",-0.4276,-0.2641,-0.4472,-0.2307
3,co,2,4,1H-triazole,1H-triazole,1H-triazole,1H-triazole,1H-triazole,1H-triazole,-1598.150651,...,3.0,1.0,1.0,1.0,-8.394981e+63,"49\n11/21/2023 17:07, XYZ structure generated ...",-0.4550,-0.2189,-0.4588,-0.2944
4,co,2,2,4H-pyran,4H-pyran,4H-pyran,4H-pyran,4H-pyran,4H-pyran,-1760.521993,...,1.0,0.0,1.0,1.0,-5.015494e+77,"73\n11/21/2023 17:07, XYZ structure generated ...",-0.4191,-0.3830,-0.4222,-0.2479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,cr,3,4,thioazole,thioazole,thioazole,thioazole,thioazole,thioazole,-3499.871514,...,3.0,1.0,1.0,1.0,-2.814787e+74,"49\n11/21/2023 17:08, XYZ structure generated ...",-0.5856,-0.4340,-0.5849,-0.4285
364,cr,3,2,thioformaldehyde,thioformaldehyde,thioformaldehyde,thioformaldehyde,thioformaldehyde,thioformaldehyde,-2710.102889,...,1.0,0.0,1.0,1.0,-4.906169e+35,"25\n11/21/2023 17:08, XYZ structure generated ...",-0.6637,-0.5461,-0.6671,-0.5366
365,cr,3,4,thioformaldehyde,thioformaldehyde,thioformaldehyde,thioformaldehyde,thioformaldehyde,thioformaldehyde,-2710.146511,...,3.0,1.0,1.0,1.0,-4.906169e+35,"25\n11/21/2023 17:08, XYZ structure generated ...",-0.6594,-0.5326,-0.6686,-0.5202
366,cr,3,2,thiophene,thiophene,thiophene,thiophene,thiophene,thiophene,-3403.325417,...,1.0,0.0,0.0,1.0,-7.745070e+69,"55\n11/21/2023 17:08, XYZ structure generated ...",-0.5532,-0.5330,-0.5601,-0.4307


In [48]:
df_raw["name"].str.split("_").str.get(2).unique()

array(['1H-tetrazole', '1H-triazole', '4H-pyran', '4H-thiopyran',
       '[NH]=[CH]-[OH]', '[PH2]-[CH2]-[OH]', '[PH2]-[NH]-[NH]-[PH2]',
       '[PH]=[CH]-[OH]', '[NH2]-[NH]-[NH]-[NH2]', '[NH2]-[O]-[O]-[NH2]',
       '[OH]-[CH]=[CH]-[OH]', 'acrylamide', 'bipyrimidine', 'bifuran',
       'dmf', 'oxazoline', 'pyridine-N-oxide', 'thiane', 'thioazole',
       'thioformaldehyde', 'thiophene'], dtype=object)

In [28]:
df_raw = pd.read_csv(raw_data_path / "composition_test" / "composition_test_data_raw.csv")
df_comp_test = pd.read_csv("/home/ralf/Dropbox (MIT)/many_body_expansion/data/new_calculations/interpolation_set/ligand_composition_data2.csv")

In [30]:
for index, row in df_raw.iterrows():
    metal, ox, lig1, lig2, lig3, lig4, lig5, lig6 = row["name"].split("_")

    print(metal, ox, lig1)
    mask = (
        (df_comp_test["metal"] == metal)
        & (df_comp_test["ox"] == int(ox))
        & (df_comp_test["lig1"] == lig1)
        & (df_comp_test["lig2"] == lig2)
        & (df_comp_test["lig3"] == lig3)
        & (df_comp_test["lig4"] == lig4)
        & (df_comp_test["lig5"] == lig5)
        & (df_comp_test["lig6"] == lig6)
    )
    assert len(df_comp_test[mask])  == 2
    # Loop over the two spins and write .xyz
    for _, spin in df_comp_test[mask].iterrows():
        with open(raw_data_path / "composition_test" / "xyzs" / f"{row['name']}_{spin['spin']}.xyz", "w") as fout:
            fout.write(spin["opt_geo"].replace("\t", "    "))

name                       co_2_hydrogencyanide_hydrogencyanide_hydrogenc...
metal                                                                     co
ox                                                                         2
high_spin                                                                  4
spin_splitting_kcal/mol                                           -13.998205
energetic_homo_ls_eV                                              -14.968983
energetic_homo_hs_eV                                              -15.268309
energetic_lumo_ls_eV                                              -10.321279
energetic_lumo_hs_eV                                              -10.432845
energetic_gap_ls_eV                                                 4.647705
energetic_gap_hs_eV                                                 4.835463
Name: 0, dtype: object
co 2 hydrogencyanide
name                       co_2_hydrogencyanide_hydrogencyanide_hydrogenc...
metal                           

In [76]:
df_raw = pd.read_csv(raw_data_path / "validation" / "validation_data_raw.csv")
df_train_val = pd.read_csv("/home/ralf/Dropbox (MIT)/many_body_expansion/data/training_data.csv")
for index, row in df_raw.iterrows():
    metal, ox, lig1, lig2, lig3, lig4, lig5, lig6 = row["name"].split("_")

    print(metal, ox, lig1)
    mask = (
        (df_train_val["metal"] == metal)
        & (df_train_val["ox"] == int(ox))
        & (df_train_val["lig1"] == lig1)
        & (df_train_val["lig2"] == lig2)
        & (df_train_val["lig3"] == lig3)
        & (df_train_val["lig4"] == lig4)
        & (df_train_val["lig5"] == lig5)
        & (df_train_val["lig6"] == lig6)
    )

    assert len(df_train_val[mask])  == 1
    # Write the two .xyz files
    entry = df_train_val[mask].iloc[0]
    with open(raw_data_path / "validation" / "xyzs" / f"{row['name']}_{entry['low_spin']}.xyz", "w") as fout:
        # Check if the first entry is a int:
        xyz_string = entry["xyz_ls"]
        try:
            n_atoms = int(xyz_string.split()[0])
            fout.write(xyz_string.replace("\t", "    "))
        except ValueError as m:
            n_atoms = len(xyz_string.split("\n")) - 1
            fout.write(f"{n_atoms}\n\n")
            fout.write(xyz_string.replace("\t", "    "))
    with open(raw_data_path / "validation" / "xyzs" / f"{row['name']}_{entry['high_spin']}.xyz", "w") as fout:
        xyz_string = entry["xyz_hs"]
        try:
            n_atoms = int(xyz_string.split()[0])
            fout.write(xyz_string.replace("\t", "    "))
        except ValueError as m:
            n_atoms = len(xyz_string.split("\n")) - 1
            fout.write(f"{n_atoms}\n\n")
            fout.write(xyz_string.replace("\t", "    "))

co 2 [SH]-[CH]=[CH]-[SH]
cr 2 [OH]-[CH2]-[CH2]-[OH]
cr 2 [PH2]-[CH2]-[CH2]-[PH2]
cr 2 [S]=[CH]-[CH]=[S]
cr 3 [PH2]-[S]-[S]-[PH2]
fe 2 [OH]-[NH]-[NH]-[OH]
fe 2 [O]=[CH]-[CH]=[O]
fe 2 [SH]-[CH]=[CH]-[SH]
fe 2 [SH]-[S]-[S]-[SH]
fe 2 [S]=[CH]-[CH]=[S]
fe 3 [OH]-[CH2]-[CH2]-[OH]
mn 2 [OH]-[NH]-[NH]-[OH]
mn 2 [SH]-[CH]=[CH]-[SH]
mn 2 [SH]-[PH]-[PH]-[SH]
mn 3 [PH2]-[CH2]-[CH2]-[PH2]
co 2 [NH]=[CH]-[CH]=[NH]
co 2 [OH]-[NH]-[NH]-[OH]
co 2 [SH]-[NH]-[NH]-[SH]
co 3 [NH]=[CH]-[CH]=[NH]
cr 3 [NH2]-[PH]-[PH]-[NH2]
cr 3 [NH2]-[S]-[S]-[NH2]
cr 3 [NH]=[CH]-[CH]=[NH]
fe 2 [NH]=[CH]-[CH]=[NH]
co 2 [O+]-[PH3-]
co 2 [O-]#[C+]
co 2 [OH]-[OH]
co 2 [OH]-[SH]
co 2 formaldehyde
co 2 phosphine
co 2 [PH]=[CH2]
co 2 [SH]-[CH3]
co 3 [O+]-[PH3-]
co 3 formaldehyde
co 3 [O]=[PH]
co 3 [PH]=[CH2]
cr 2 [NH2]-[OH]
cr 2 [NH]=[NH]
cr 2 [N]#[N]
cr 2 [OH]-[PH2]
cr 2 formaldehyde
cr 2 [PH]=[CH2]
cr 3 [NH]=[NH]
cr 3 [O+]-[NH3-]
fe 2 [NH]=[NH]
fe 2 [O+]-[NH3-]
fe 2 formaldehyde
fe 2 [O]=[PH]
fe 2 [PH]=[CH2]
fe 2 [SH]-[CH3]
fe 2 

In [77]:
df_raw = pd.read_csv(raw_data_path / "training" / "training_data_raw.csv")
df_train_val = pd.read_csv("/home/ralf/Dropbox (MIT)/many_body_expansion/data/training_data.csv")
for index, row in df_raw.iterrows():
    metal, ox, lig1, lig2, lig3, lig4, lig5, lig6 = row["name"].split("_")

    print(metal, ox, lig1)
    mask = (
        (df_train_val["metal"] == metal)
        & (df_train_val["ox"] == int(ox))
        & (df_train_val["lig1"] == lig1)
        & (df_train_val["lig2"] == lig2)
        & (df_train_val["lig3"] == lig3)
        & (df_train_val["lig4"] == lig4)
        & (df_train_val["lig5"] == lig5)
        & (df_train_val["lig6"] == lig6)
    )

    assert len(df_train_val[mask])  == 1
    # Write the two .xyz files
    entry = df_train_val[mask].iloc[0]
    with open(raw_data_path / "training" / "xyzs" / f"{row['name']}_{entry['low_spin']}.xyz", "w") as fout:
        # Check if the first entry is a int:
        xyz_string = entry["xyz_ls"]
        try:
            n_atoms = int(xyz_string.split()[0])
            fout.write(xyz_string.replace("\t", "    "))
        except ValueError as m:
            n_atoms = len(xyz_string.split("\n")) - 1
            fout.write(f"{n_atoms}\n\n")
            fout.write(xyz_string.replace("\t", "    "))
    with open(raw_data_path / "training" / "xyzs" / f"{row['name']}_{entry['high_spin']}.xyz", "w") as fout:
        xyz_string = entry["xyz_hs"]
        try:
            n_atoms = int(xyz_string.split()[0])
            fout.write(xyz_string.replace("\t", "    "))
        except ValueError as m:
            n_atoms = len(xyz_string.split("\n")) - 1
            fout.write(f"{n_atoms}\n\n")
            fout.write(xyz_string.replace("\t", "    "))

co 2 [OH]-[PH]-[PH]-[OH]
co 2 [OH]-[S]-[S]-[OH]
co 2 [O]=[CH]-[CH]=[O]
co 2 [PH2]-[CH]=[CH]-[PH2]
co 2 [PH2]-[PH]-[PH]-[PH2]
co 2 [PH2]-[S]-[S]-[PH2]
co 2 [SH]-[CH2]-[CH2]-[SH]
co 2 [SH]-[PH]-[PH]-[SH]
co 2 [SH]-[S]-[S]-[SH]
co 2 [S]=[CH]-[CH]=[S]
co 3 [O]=[CH]-[CH]=[O]
co 3 [S]=[CH]-[CH]=[S]
cr 2 [OH]-[PH]-[PH]-[OH]
cr 2 [OH]-[P]=[P]-[OH]
cr 2 [PH2]-[CH]=[CH]-[PH2]
cr 2 [PH2]-[O]-[O]-[PH2]
cr 2 [PH2]-[PH]-[PH]-[PH2]
cr 2 [SH]-[CH2]-[CH2]-[SH]
cr 2 [SH]-[NH]-[NH]-[SH]
cr 2 [SH]-[PH]-[PH]-[SH]
cr 2 [SH]-[S]-[S]-[SH]
cr 2 [S]=[P]-[P]=[S]
cr 3 [NH]=[P]-[P]=[NH]
cr 3 [OH]-[PH]-[PH]-[OH]
cr 3 [O]=[CH]-[CH]=[O]
cr 3 [PH2]-[O]-[O]-[PH2]
fe 2 [OH]-[PH]-[PH]-[OH]
fe 2 [OH]-[S]-[S]-[OH]
fe 2 [PH2]-[S]-[S]-[PH2]
fe 2 [SH]-[CH2]-[CH2]-[SH]
fe 2 [SH]-[NH]-[NH]-[SH]
fe 2 [SH]-[PH]-[PH]-[SH]
fe 3 [O]=[CH]-[CH]=[O]
fe 3 [PH2]-[CH]=[CH]-[PH2]
fe 3 [S]=[CH]-[CH]=[S]
mn 2 [OH]-[S]-[S]-[OH]
mn 2 [O]=[CH]-[CH]=[O]
mn 2 [PH2]-[PH]-[PH]-[PH2]
mn 2 [PH2]-[S]-[S]-[PH2]
mn 2 [SH]-[CH2]-[CH2]-[SH]
mn 2 [SH]-[NH]